In [1]:
%matplotlib notebook
from astropy.table import Table, vstack, join
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
from astropy.io.fits import getheader
from utilities import redshifts_from_papers

import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

/home/boada/Projects/swiftXRT/paper/utilities.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def load_detcats(outpath, fields):
    for field in fields:
        if os.path.exists(f'{outpath}/{field}/{field}_vtp.detect'):
            data_part = Table.read(f'{outpath}/{field}/{field}_vtp.detect', format='fits')
            if len(data_part) > 0:
                data_part['Field'] = field
                try:
                    hdr = getheader(f'{outpath}/{field}/{field}_vtp_high.detect', ext=1)
                except FileNotFoundError:
                    continue
                data_part['Field_EXP'] = hdr['EXPOSURE']
        else:
            continue

        # handle the first pass when we only have one part
        try:
            data = vstack([data, data_part])
        except NameError:
            data = data_part
        
    return data


def load_MCMCfits(outpath, fields):
    for field in fields:
        if os.path.exists(f'{outpath}/{field}/{field}_mcmcfits.txt'):
            try:
                fit_part = Table.read(f'{outpath}/{field}/{field}_mcmcfits.txt', format='ascii', header_start=0)
            except :
                print(field)
                raise (FileNotFoundError)
            src_part = Table.read(f'{outpath}/{field}/{field}_vtp.detect', format='fits')
            
            fit_part.rename_column('ID', 'INDEX')
            try:
                data_part = join(fit_part, src_part, keys='INDEX', join_type='inner')
            except ValueError:
                continue
        else:
            continue

        # handle the first pass when we only have one part
        try:
            data = vstack([data, data_part])
        except NameError:
            data = data_part
        
    return data

def load_spectralfits(outpath, fields):
    for field in fields:
        if os.path.exists(f'{outpath}/{field}/spec_files/{field}_xspec.results'):
            try:
                data_part = Table.read(f'{outpath}/{field}/spec_files/{field}_xspec.results', 
                                       format='ascii', header_start=0)
                if len(data_part) < 1:
                    continue
                
            except ValueError:
                continue
        else:
            continue

        # handle the first pass when we only have one part
        try:
            data = vstack([data, data_part])
        except NameError:
            data = data_part
        
    return data

In [3]:
# get file data
data = load_PSZcatalog()
data = data.sort_values('NAME')
data = redshifts_from_papers(data)

datapath = '../data_full_new'
data.replace(-1, np.nan, inplace=True)

fields = [n.replace(' ', '_') for n in data['NAME']]

fits = load_MCMCfits(datapath, fields) # not currently used, but gives access to beta-model params
cats = load_detcats(datapath, fields)
specs = load_spectralfits(datapath, fields)

# check to see which fields have XRT data
field_mask = [True if os.path.isdir(f'{datapath}/{n.replace(" ", "_")}') else False for n in data['NAME']]
                                    
data['SWIFT'] = False
data['SWIFT'] = field_mask

/home/boada/Projects/swiftXRT/paper/utilities.py:215: TableReplaceWarning: converted column 'Nspec' from integer to float
  data_barrena = load_extraCatalogs('barrena2018', merge=True)
/home/boada/Projects/swiftXRT/paper/utilities.py:215: TableReplaceWarning: converted column 'R' from integer to float
  data_barrena = load_extraCatalogs('barrena2018', merge=True)
/home/boada/Projects/swiftXRT/paper/utilities.py:216: TableReplaceWarning: converted column 'N' from integer to float
  data_aguado = load_extraCatalogs('aguado-barahona2019', merge=True)
/home/boada/Projects/swiftXRT/paper/utilities.py:218: TableReplaceWarning: converted column 'ID' from integer to float
  data_zohren = load_extraCatalogs('zohren2019', merge=True)


In [4]:
# add more columns -- don't think I really need these in the end.
# data['XRT_exp'] = np.nan
# data['N_H'] = np.nan

In [5]:
cats.remove_column('R')
df_cats = cats.to_pandas()
df_cats.Field = df_cats.Field.str.replace('_', ' ')

df_specs = specs.to_pandas()
df_specs.FIELD = df_specs.FIELD.str.replace('_', ' ')

fits.remove_column('R')
df_fits = fits.to_pandas()

In [6]:
# df_fits

In [7]:
# df_cats

In [8]:
# df_specs

In [9]:
df_data = data[['NAME', 'INDEX_PSZ2', 'RA', 'DEC', 'REDSHIFT']]

In [10]:
# df_data

In [11]:
# merge the catalog information to the main "data" frame
df_m = df_data.merge(df_cats,                                                      
               how='inner',                                              
               left_on='NAME',                                          
               right_on='Field',                                           
               suffixes=('_data', '_detc'))

# merge the spectral information to this new merged frame
df_m2 = df_m.merge(df_specs,                                                      
               how='left',                                              
               left_on=['NAME', 'INDEX'],                                          
               right_on=['FIELD', 'INDEX'],                                           
               suffixes=('_data', '_detc'))

In [12]:
# df_m

In [13]:
# df_m2

In [14]:
# only select extended objects that are within 6' of the PSZ position
df_close_ext = df_m2[(df_m2.PSZ_dist < 6) & (df_m2.Extended == 1)]

In [15]:
df_close_ext.columns

Index(['NAME', 'INDEX_PSZ2', 'RA_data', 'DEC_data', 'REDSHIFT', 'INDEX',
       'RA_VTP', 'RA_ERR_VTP', 'DEC_VTP', 'DEC_ERR_VTP', 'X_VTP', 'Y_VTP',
       'X_ERR_VTP', 'Y_ERR_VTP', 'SRC_AREA', 'NET_COUNTS', 'NET_COUNTS_ERR',
       'BKG_COUNTS', 'BKG_COUNTS_ERR', 'NET_RATE', 'NET_RATE_ERR', 'BKG_RATE',
       'BKG_RATE_ERR', 'EXPTIME', 'SRC_CUTOFF', 'FSP', 'EDGE_OF_FIELD',
       'SHAPE', 'ROTANG', 'COMPONENT', 'HIGH', 'X', 'Y', 'X_ERR', 'Y_ERR',
       'RA_detc', 'DEC_detc', 'RA_ERR', 'DEC_ERR', 'PSZ_dist', 'Extended',
       'Field', 'Field_EXP', 'SN', 'redCHI2', 'DOF', 'NORM', 'NORM_NEG',
       'NORM_POS', 'FLUX', 'CNT_RATE_NET', 'CNT_RATE_NET_ERR',
       'CNT_RATE_MODEL', 'ECF', 'LUM', 'FLUX_CORR', 'LUM_CORR', 'CORR',
       'FIELD'],
      dtype='object')

In [16]:
cols = ['NAME', 'RA_detc', 'DEC_detc', 'REDSHIFT', 'INDEX', 'PSZ_dist', 'EXPTIME',
        'SN', 'redCHI2', 'DOF', 'NORM', 'NORM_NEG', 'NORM_POS', 'FLUX', 'CNT_RATE_NET', 
        'CNT_RATE_MODEL', 'ECF', 'LUM', 'FLUX_CORR', 'LUM_CORR', 'CORR']

In [17]:
df_close_ext[df_close_ext.FLUX_CORR.isnull()][cols]

,NAME,RA_detc,DEC_detc,REDSHIFT,INDEX,PSZ_dist,EXPTIME,SN,redCHI2,DOF,...,NORM_NEG,NORM_POS,FLUX,CNT_RATE_NET,CNT_RATE_MODEL,ECF,LUM,FLUX_CORR,LUM_CORR,CORR
111,PSZ1 G292.00-43.64,38.184324,-71.219107,NaN,9,3.472973,327940.125000,-1.0,11.6202,821.0,...,-1.0,-1.0,1.736700e-12,0.06475,0.072833,2.384500e-11,4.206300e+43,NaN,NaN,NaN
114,PSZ1 G292.00-43.64,38.157531,-71.263793,NaN,12,0.766725,322966.812500,-1.0,11.6006,821.0,...,-1.0,-1.0,1.743000e-12,0.06490,0.073110,2.384100e-11,4.221400e+43,NaN,NaN,NaN
128,PSZ1 G292.00-43.64,37.905434,-71.333463,NaN,26,5.990574,314961.250000,-1.0,11.6219,821.0,...,-1.0,-1.0,1.751100e-12,0.06512,0.073457,2.383800e-11,4.241000e+43,NaN,NaN,NaN
131,PSZ1 G292.00-43.64,37.872597,-71.293228,NaN,29,5.647563,344272.687500,-1.0,11.6324,821.0,...,-1.0,-1.0,1.745000e-12,0.06502,0.073209,2.383600e-11,4.226200e+43,NaN,NaN,NaN
750,PSZ2 G283.91+73.87,187.478366,11.768358,0.0852,4,4.800089,9156.241211,-1.0,2.0489,368.0,...,-1.0,-1.0,1.764900e-11,0.75180,0.738245,2.390700e-11,3.059400e+44,NaN,NaN,NaN


In [18]:
# calculate the errors for the flux and Luminosity
for c in ['FLUX_CORR_POSERR', 'FLUX_CORR_NEGERR', 'LUM_CORR_POSERR', 'LUM_CORR_NEGERR']:
    df_close_ext.loc[:, c] = np.nan

df_close_ext['FLUX_CORR_POSERR'] = df_close_ext['FLUX_CORR'] / df_close_ext['NORM'] * df_close_ext['NORM_POS']
df_close_ext['FLUX_CORR_NEGERR'] = df_close_ext['FLUX_CORR'] / df_close_ext['NORM'] * df_close_ext['NORM_NEG']
df_close_ext['LUM_CORR_POSERR'] = df_close_ext['LUM_CORR'] / df_close_ext['NORM'] * df_close_ext['NORM_POS']
df_close_ext['LUM_CORR_NEGERR'] = df_close_ext['LUM_CORR'] / df_close_ext['NORM'] * df_close_ext['NORM_NEG']

/home/boada/.local/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/boada/.local/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/boada/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [19]:
df_close_ext

,NAME,INDEX_PSZ2,RA_data,DEC_data,REDSHIFT,INDEX,RA_VTP,RA_ERR_VTP,DEC_VTP,DEC_ERR_VTP,...,ECF,LUM,FLUX_CORR,LUM_CORR,CORR,FIELD,FLUX_CORR_POSERR,FLUX_CORR_NEGERR,LUM_CORR_POSERR,LUM_CORR_NEGERR
0,PSZ1 G023.53-36.53,NaN,313.528542,-22.841100,0.2840,1,313.521441,0.001330,-22.811574,0.001614,...,2.166100e-11,1.341500e+44,6.396498e-13,1.453750e+44,0.083675,PSZ1 G023.53-36.53,7.672155e-14,7.672155e-14,1.743672e+43,1.743672e+43
22,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300,22,256.352326,0.000212,11.435420,0.000468,...,1.994100e-11,2.483300e+41,3.191675e-15,4.568762e+41,0.839794,PSZ1 G031.41+28.75,3.191675e-15,3.191675e-15,4.568762e+41,4.568762e+41
33,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300,33,256.284234,0.000194,11.378536,0.000146,...,2.253900e-11,6.724300e+41,6.417292e-15,9.186120e+41,0.366108,PSZ1 G031.41+28.75,1.233011e-15,1.233011e-15,1.765011e+41,1.765011e+41
39,PSZ1 G055.72+17.58,NaN,276.342547,27.804647,0.1940,1,276.379384,0.001091,27.741733,0.000991,...,2.077600e-11,1.336600e+44,1.451761e-12,1.436287e+44,0.074582,PSZ1 G055.72+17.58,9.712849e-14,9.712849e-14,9.609322e+42,9.609322e+42
40,PSZ1 G057.42-10.77,NaN,304.356094,16.092529,0.1360,1,304.366947,0.000883,16.062207,0.000763,...,2.102400e-11,8.614700e+43,1.959528e-12,9.071281e+43,0.053000,PSZ1 G057.42-10.77,1.279963e-13,1.279963e-13,5.925356e+42,5.925356e+42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,PSZ2 G358.34-47.31,1647.0,323.485677,-42.666426,0.2000,3,323.500210,0.000547,-42.646546,0.000361,...,2.103200e-11,8.794300e+43,8.985793e-13,9.498962e+43,0.080127,PSZ2 G358.34-47.31,4.039392e-14,4.039392e-14,4.270077e+42,4.270077e+42
913,PSZ2 G358.34-47.31,1647.0,323.485677,-42.666426,0.2000,4,323.423754,0.001739,-42.727664,0.000935,...,2.236000e-11,5.171300e+42,7.320834e-14,7.738962e+42,0.496522,PSZ2 G358.34-47.31,1.659103e-14,1.659103e-14,1.753863e+42,1.753863e+42
915,PSZ2 G358.98-67.26,1650.0,348.926557,-37.779104,0.1786,1,348.934928,0.002054,-37.793994,0.001341,...,2.088400e-11,1.408900e+44,1.806741e-12,1.497099e+44,0.062601,PSZ2 G358.98-67.26,1.865201e-13,1.865201e-13,1.545540e+43,1.545540e+43
916,PSZ2 G359.07-32.12,1651.0,303.018900,-41.486882,0.1496,1,303.146133,0.000335,-41.480888,0.000363,...,2.079800e-11,2.028600e+42,3.911258e-14,2.217298e+42,0.093019,PSZ2 G359.07-32.12,7.126681e-15,7.126681e-15,4.040126e+41,4.040126e+41


In [20]:
cols = ['NAME', 'RA_detc', 'DEC_detc', 'REDSHIFT', 'INDEX', 'PSZ_dist', 'EXPTIME',
        'SN', 'redCHI2', 'DOF', 'NORM', 'NORM_NEG', 'NORM_POS', 'FLUX', 'CNT_RATE_NET', 
        'CNT_RATE_MODEL', 'ECF', 'LUM', 'FLUX_CORR', 'FLUX_CORR_POSERR', 'FLUX_CORR_NEGERR', 
        'LUM_CORR', 'LUM_CORR_POSERR', 'LUM_CORR_NEGERR', 'CORR']

t1 = Table.from_pandas(df_close_ext[cols])

In [21]:
t1.filled(-1.0)

NAME,RA_detc,DEC_detc,REDSHIFT,INDEX,PSZ_dist,EXPTIME,SN,redCHI2,DOF,NORM,NORM_NEG,NORM_POS,FLUX,CNT_RATE_NET,CNT_RATE_MODEL,ECF,LUM,FLUX_CORR,FLUX_CORR_POSERR,FLUX_CORR_NEGERR,LUM_CORR,LUM_CORR_POSERR,LUM_CORR_NEGERR,CORR
str18,float64,float64,float64,int64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
PSZ1 G023.53-36.53,313.5192459279641,-22.811192599451434,0.2840000092983246,1,1.8666280661228891,2716.9482,8.3373,0.9582,14.0,0.00150664,0.000180711,0.000180711,5.9026e-13,0.02589,0.0272495,2.1661e-11,1.3415e+44,6.396498125489376e-13,7.672155078554338e-14,7.672155078554338e-14,1.4537495739748584e+44,1.7436716087623494e+43,1.7436716087623494e+43,0.08367467310835514
PSZ1 G031.41+28.75,256.3524228060791,11.435045572929392,0.23,22,5.466414508179784,216357.27,-1.0,2153.555,1023.0,-1.0,-1.0,-1.0,1.7348e-15,5.274e-05,8.69947e-05,1.9941e-11,2.4833e+41,3.191675408206082e-15,3.191675408206082e-15,3.191675408206082e-15,4.568761552454556e+41,4.568761552454556e+41,4.568761552454556e+41,0.8397944478937526
PSZ1 G031.41+28.75,256.2842552229166,11.378437505810556,0.23,33,5.074730659124593,197958.08,5.2046,0.497,3.0,1.12045e-05,2.15282e-06,2.15282e-06,4.6975e-15,0.0001765,0.000208414,2.2539e-11,6.7243e+41,6.4172921808681235e-15,1.2330112858955345e-15,1.2330112858955345e-15,9.186119810923155e+41,1.7650107056407324e+41,1.7650107056407324e+41,0.3661079682529266
PSZ1 G055.72+17.58,276.3803066233068,27.741361425895942,0.1940000057220459,1,4.293797561529516,3424.389,14.9468,1.115,33.0,0.00307553,0.000205765,0.000205765,1.351e-12,0.05757,0.0650264,2.0776e-11,1.3366e+44,1.4517609216606323e-12,9.712849038881103e-14,9.712849038881103e-14,1.436286934042636e+44,9.609322002493325e+42,9.609322002493325e+42,0.07458247347197065
PSZ1 G057.42-10.77,304.36687269578346,16.062365346049575,0.136,1,1.9135608557965353,2917.1406,15.3093,0.6737,37.0,0.00392015,0.000256064,0.000256064,1.8609e-12,0.06928,0.0885129,2.1024e-11,8.6147e+43,1.9595281908831966e-12,1.279962824561088e-13,1.279962824561088e-13,9.071281372454982e+43,5.925356410740181e+42,5.925356410740181e+42,0.053000263788057785
PSZ1 G083.35+76.41,203.70014715336345,37.18210039979622,-1.0,1,0.9253717082144188,46343.074,-1.0,6.03,303.0,-1.0,-1.0,-1.0,3.726e-12,0.2165,0.157516,2.3655e-11,9.0241e+43,3.777180184516625e-12,3.777180184516625e-12,3.777180184516625e-12,9.148054670718324e+43,9.148054670718324e+43,9.148054670718324e+43,0.01373595934423665
PSZ1 G083.35+76.41,203.6204264527387,37.209131244254884,-1.0,2,3.3910404629742787,52051.797,5.767,0.3149,347.0,0.00041452,7.18776e-05,7.18776e-05,2.0688e-13,0.01279,0.00903249,2.2904e-11,5.0106e+42,3.0052808361136133e-13,5.211144789777088e-14,5.211144789777088e-14,7.27874137540162e+42,1.2621308045077861e+42,1.2621308045077861e+42,0.4526686176109887
PSZ1 G103.50+31.36,264.23998368261863,72.56058825187006,-1.0,1,0.982848402974421,2846.7449,-1.0,2.21,4.0,-1.0,-1.0,-1.0,1.3071e-13,0.009879,0.00613505,2.1305e-11,3.1658e+42,1.7648910526387558e-13,1.7648910526387558e-13,1.7648910526387558e-13,4.274571260380822e+42,4.274571260380822e+42,4.274571260380822e+42,0.3502341463076701
PSZ1 G103.94+25.81,283.03587113474066,72.98968469360932,0.0989999994635582,1,1.7836576822735373,2893.2185,8.5298,0.8227,15.0,0.00124087,0.000145475,0.000145475,6.2018e-13,0.02952,0.0293218,2.1151e-11,1.4703e+43,6.994488080873298e-13,8.200078602633983e-14,8.200078602633983e-14,1.6582275831706943e+43,1.9440445627806035e+42,1.9440445627806035e+42,0.1278158084545291


In [22]:
# t1.filled(-1.0).write('results_table_full_0.5_2.0.fits', overwrite=True)